In [3]:
import heapq

# Goal state
GOAL_STATE = ((1, 2, 3), (8, 0, 4), (7, 6, 5))

# Directions for movement (up, down, left, right)
MOVES = [(-1, 0), (1, 0), (0, -1), (0, 1)]


class PuzzleState:
    def __init__(self, puzzle, parent=None, move=None, g=0):
        self.puzzle = puzzle  # 2D tuple (state of the puzzle)
        self.parent = parent  # Parent PuzzleState
        self.move = move  # Move that led to this state
        self.g = g  # Cost to reach this state (g-value)
        self.h = self.manhattan_distance()  # Heuristic (h-value)
        self.f = g + self.h  # f = g + h (total cost)

    def manhattan_distance(self):
        """Calculate the Manhattan distance heuristic."""
        distance = 0
        for r in range(3):
            for c in range(3):
                value = self.puzzle[r][c]
                if value == 0:
                    continue
                goal_r, goal_c = divmod(value - 1, 3)
                distance += abs(r - goal_r) + abs(c - goal_c)
        return distance

    def get_possible_moves(self):
        """Generate possible moves (up, down, left, right) from the current state."""
        empty_r, empty_c = [(r, c) for r in range(3) for c in range(3) if self.puzzle[r][c] == 0][0]
        for dr, dc in MOVES:
            new_r, new_c = empty_r + dr, empty_c + dc
            if 0 <= new_r < 3 and 0 <= new_c < 3:
                new_puzzle = list(list(row) for row in self.puzzle)  # Create a mutable copy of the puzzle
                new_puzzle[empty_r][empty_c], new_puzzle[new_r][new_c] = new_puzzle[new_r][new_c], new_puzzle[empty_r][empty_c]
                yield PuzzleState(tuple(tuple(row) for row in new_puzzle), parent=self, move=(new_r, new_c), g=self.g + 1)

    def __lt__(self, other):
        """For the priority queue to sort PuzzleState objects by f-value."""
        return self.f < other.f

    def __repr__(self):
        return f"PuzzleState(puzzle={self.puzzle}, f={self.f}, g={self.g}, h={self.h})"


def a_star(start_state):
    """Solve the 8-puzzle problem using the A* algorithm."""
    open_list = []
    closed_list = set()
    heapq.heappush(open_list, start_state)

    while open_list:
        current_state = heapq.heappop(open_list)

        # If we reach the goal state, reconstruct the solution
        if current_state.puzzle == GOAL_STATE:
            solution = []
            while current_state:
                solution.append(current_state)
                current_state = current_state.parent
            return solution[::-1]  # Return the solution in the correct order

        closed_list.add(current_state.puzzle)

        # Generate possible moves and add them to the open list
        for next_state in current_state.get_possible_moves():
            if next_state.puzzle not in closed_list:
                heapq.heappush(open_list, next_state)

    return None  # No solution found


def print_solution(solution):
    """Print the sequence of moves to reach the goal."""
    for state in solution:
        for row in state.puzzle:
            print(row)
        print()


if __name__ == "__main__":
    # New initial puzzle state (this state is different and more scrambled to show multiple steps)
    start_state = PuzzleState(((1, 2, 3), (8, 6, 4), (7, 5, 0)))

    solution = a_star(start_state)

    if solution:
        print("Solution found!")
        print_solution(solution)
    else:
        print("No solution found.")

Solution found!
(1, 2, 3)
(8, 6, 4)
(7, 5, 0)

(1, 2, 3)
(8, 6, 4)
(7, 0, 5)

(1, 2, 3)
(8, 0, 4)
(7, 6, 5)

